In [1]:
%config IPCompleter.greedy=True

In [2]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup

In [6]:
rawHTML = get('https://www.infobae.com/politica/2018/11/21/miguel-angel-pichetto-no-soy-el-bolsonaro-argentino/')

In [11]:
html = BeautifulSoup(rawHTML.content, 'html.parser')

In [77]:
html.prettify()

'<!DOCTYPE html>\n<html class=" story " itemscope="" itemtype="http://schema.org/ItemList">\n <head>\n  <link href="/pb/resources/favicon.ico" rel="shortcut icon" type="image/x-icon"/>\n  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>\n  <meta charset="utf-8">\n   <meta content="width=device-width, initial-scale=1.0, user-scalable=yes, minimum-scale=0.5, maximum-scale=2.0" id="viewport" name="viewport"/>\n   <meta content="" property="keywords"/>\n   <meta 35="" años="" en="" esto""="" estoy="" hace="" itemprop="description" name="description" que="" value="El senador se diferenció del presidente electo de Brasil. Y remarcó: ">\n    <meta content="" name="news_keywords"/>\n    <meta name="twitter:site" value="@infobae"/>\n    <meta content="summary_large_image" name="twitter:card"/>\n    <meta content="@infobae" name="twitter:creator"/>\n    <meta 35="" años="" content="El senador se diferenció del presidente electo de Brasil. Y remarcó: " en="" esto""="" estoy="" hace

In [79]:
import numpy as np

title = html.find('title').text
shorttext = html.find(attrs={'class':'subheadline'}).text
date = html.find(attrs={'class':'byline-date'}).text
fulltext = ""
tags = []

p_all = set(html.find_all('p'))
p_to_remove = set(html.find_all('p', attrs={"class":""}))

p_needed = p_all - p_to_remove

for p in p_needed:
    fulltext = fulltext + p.text
    
for story_tag in html.find_all(class_='story-tag'):
    tags.append(story_tag.text)

#print(title)
#print(fulltext)
#print(shorttext)
#print(tags)
#print(date)

In [80]:
import re
from datetime import datetime

dayRange = re.search("\d", date)
year = date[-5:]

if "Enero" in date: month = "01"
if "Febrero" in date: month = "02"
if "Marzo" in date: month = "03"
if "Abril" in date: month = "04"
if "Mayo" in date: month = "05"
if "Junio" in date: month = "06"
if "Julio" in date: month = "07"
if "Agosto" in date: month = "08"
if "Septiembre" in date: month = "09"
if "Octubre" in date: month = "10"
if "Noviembre" in date: month = "11"
if "Diciembre" in date: month = "12"
    
correctedDate = datetime(int(year), int(month), int(date[dayRange.start() : dayRange.end() + 1]),3,0,0)
print(correctedDate)


2018-11-21 03:00:00


In [81]:
import pymongo

myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["DiarioStage"]
mycol = mydb.Infobae

mykey = {"title": title}
mydict = {"fulltext": fulltext, "shorttext": shorttext, "tags": tags, "date": correctedDate}

x = mycol.update_one(mykey, {'$set' : mydict}, upsert=True)